In [1]:
import os
import numpy as np
os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.ui.port=4041 --packages org.apache.kafka:kafka_2.11:0.9.0.0,org.apache.kafka:kafka-clients:0.9.0.0  pyspark-shell'

In [2]:
from pyspark import SparkContext
sc = SparkContext("local[1]", "KafkaSendStream") 
from kafka import KafkaProducer
import time

In [3]:
import json
from satori.rtm.client import make_client, SubscriptionMode
from math import log

# Make a proxy class

endpoint = "wss://open-data.api.satori.com"
appkey = "dC6c33Fbb5ECdAC1Ef2aB77dcBfBB0B0"
channel = "cryptocurrency-market-data"

with make_client(endpoint=endpoint, appkey=appkey) as client:
    print('Connected to Satori RTM!')
    messagebox=[]

    class SubscriptionObserver(object):
        def on_subscription_data(self, data):
            for message in data['messages']:
                messagebox.append(message)

    subscription_observer = SubscriptionObserver()
    client.subscribe(
        channel,
        SubscriptionMode.SIMPLE,
        subscription_observer,
        {'filter':'select * from `cryptocurrency-market-data` where exchange = "Bitstamp" and basecurrency = "USD" and cryptocurrency="BTC"'})
        #  exchange != "ShapheShift" OU type != "ratequote" si on laisse ShapeShift

    producer = KafkaProducer(bootstrap_servers='localhost:9092')
    i=0
    numOfColumnsX=3   # number of columns of X counting t_0 column with only ones
    numOfRendements=10
    timeBetweenPrices=3
    sumOfProxies=np.zeros(numOfColumnsX)
    sumOfSquaredProxies=np.zeros(numOfColumnsX)
    while not messagebox: # waiting for a first price to arrive
        time.sleep(1)
    firstMessage=messagebox.pop()
    firstMessage=json.dumps(firstMessage, ensure_ascii=False)
    firstMessage=json.loads(firstMessage)
    lastPrice=float(firstMessage['price']) # get that first price
    lastTime=int(firstMessage['timestamp'])
    rendements=np.empty(0)
    proxies=np.empty(0)
    interpolatedTimes=[]
    interpolatedPrices=[]
    while True:
        proxies=proxies[1:]
        while proxies.size<numOfColumnsX: # Pas -1 car le dernier proxy sera utilisé comme y            
            if messagebox and not interpolatedTimes:
                newMessage=messagebox.pop()
                newMessage=json.dumps(newMessage, ensure_ascii=False)
                newMessage=json.loads(newMessage)
                prices=[lastPrice, float(newMessage['price'])]
                times=[lastTime, int(newMessage['timestamp'])]
                interpolatedTimes=list(range(lastTime, int(newMessage['timestamp']), timeBetweenPrices))
                interpolatedPrices=list(np.interp(interpolatedTimes, times, prices))
            while interpolatedTimes and proxies.size<numOfColumnsX:
                newPrice=interpolatedPrices.pop(0)
                rendement=np.log(newPrice/lastPrice)
                lastPrice=newPrice
                lastTime=interpolatedTimes.pop(0)
                rendements=np.append(rendements, rendement)
                if rendements.size==numOfRendements:
                    proxies=np.append(proxies,np.sum(np.square(rendements))/numOfRendements)
                    rendements=rendements[1:]
            time.sleep(1)
        
        sumOfProxies+=proxies
        sumOfSquaredProxies+=proxies**2
        mean=sumOfProxies/(i+1)
        std=np.sqrt(sumOfSquaredProxies/(i+1)-mean**2) # https://math.stackexchange.com/a/1379804
        print(mean,std)
        if i!=0:
            y=(proxies[-1]-mean[-1])/std[-1]
            message=np.array2string(np.append([i,y], np.insert((proxies[:numOfColumnsX-1]-mean[:numOfColumnsX-1])/std[:numOfColumnsX-1],0,1)),separator=",") # Ajout de la colonne t_0 aux proxies 
            producer.send('volatility', message)
            print(message)
        i=i+1

Connected to Satori RTM!
(array([  1.73367335e-09,   1.75167704e-09,   1.41582189e-09]), array([ 0.,  0.,  0.]))
(array([  1.74267520e-09,   1.58374947e-09,   4.63762729e-09]), array([  9.00184691e-12,   1.67927579e-10,   3.22180540e-09]))
[ 1., 1., 1., 1.,-1.]
(array([  1.63372409e-09,   3.67564387e-09,   7.86059697e-09]), array([  1.54255334e-10,   2.96156111e-09,   5.26261221e-09]))
[ 2.        , 1.22485548, 1.        ,-1.41260727, 1.41269711]
(array([  3.19015124e-09,   6.33336699e-09,   1.10847316e-08]), array([  2.69911882e-09,   5.26959293e-09,   7.20808216e-09]))
[ 3.        , 1.34188315, 1.        , 1.72992808, 1.51305223]
(array([  5.41342826e-09,   9.21812072e-09,   1.43100320e-08]), array([  5.05964789e-09,   7.44997342e-09,   9.12005524e-09]))
[ 4.        , 1.41459684, 1.        , 1.75765355, 1.54886659]
(array([  7.97071282e-09,   1.22169728e-08,   1.75925070e-08]), array([  7.35063845e-09,   9.55077603e-09,   1.10989211e-08]))
[ 5.        , 1.47873607, 1.        , 1.7394

(array([  2.47908205e-08,   2.51996649e-08,   2.56105274e-08]), array([  1.06597866e-08,   1.00977766e-08,   9.48451566e-09]))
[ 43.        , -0.60950908,  1.        , -0.47394562, -0.54238095]
(array([  2.46781984e-08,   2.50803307e-08,   2.54844366e-08]), array([  1.05671189e-08,   1.00162762e-08,   9.41576152e-09]))
[ 44.        , -0.58922411,  1.        , -0.46894288, -0.52421695]
(array([  2.45727947e-08,   2.49685071e-08,   2.53661503e-08]), array([  1.04755177e-08,   9.93516468e-09,   9.34659687e-09]))
[ 45.        , -0.56949982,  1.        , -0.45278581, -0.50649026]
(array([  2.44741489e-08,   2.48637147e-08,   2.52551704e-08]), array([  1.03850507e-08,   9.85456666e-09,   9.27721587e-09]))
[ 46.        , -0.55028093,  1.        , -0.43694596, -0.48915897]
(array([  2.43818388e-08,   2.47655143e-08,   2.51513705e-08]), array([  1.02957715e-08,   9.77458661e-09,   9.20760964e-09]))
[ 47.        , -0.52984397,  1.        , -0.42139375, -0.47218559]
(array([  2.42954767e-08,   2.

(array([  1.73326574e-08,   1.73143253e-08,   1.72957974e-08]), array([  1.30302565e-08,   1.30532911e-08,   1.30764910e-08]))
[ 86.        , -1.31197632,  1.        , -1.31960893, -1.3158017 ]
(array([  1.71372724e-08,   1.71191597e-08,   1.71008533e-08]), array([  1.30835557e-08,   1.31059519e-08,   1.31285100e-08]))
[ 87.        , -1.29185524,  1.        , -1.29922555, -1.29554946]
(array([  1.69462890e-08,   1.69283907e-08,   1.69103009e-08]), array([  1.31326248e-08,   1.31544132e-08,   1.31763603e-08]))
[ 88.        , -1.27262861,  1.        , -1.2797547 , -1.27620073]
(array([  1.67595605e-08,   1.67418717e-08,   1.67236875e-08]), array([  1.31777373e-08,   1.31989463e-08,   1.32206954e-08]))
[ 89.        , -1.2562573 ,  1.        , -1.26113014, -1.25769009]
(array([  1.65769465e-08,   1.65591599e-08,   1.65408725e-08]), array([  1.32191450e-08,   1.32401772e-08,   1.32617483e-08]))
[ 90.        , -1.24066172,  1.        , -1.24329211, -1.24198244]
(array([  1.63980133e-08,   1.

(array([  1.67275862e-08,   1.68112433e-08,   1.68705712e-08]), array([  1.16730123e-08,   1.16034191e-08,   1.15455378e-08]))
[  1.27000000e+02, -6.51760013e-01,  1.00000000e+00, -1.01809078e-01,
  -3.76567129e-01]
(array([  1.66943629e-08,   1.67533705e-08,   1.67882460e-08]), array([  1.16337539e-08,   1.15768875e-08,   1.15383546e-08]))
[ 128.        ,  -0.91326982,   1.        ,  -0.36553858,  -0.6398714 ]
(array([  1.66378344e-08,   1.66725801e-08,   1.66833877e-08]), array([  1.16066936e-08,   1.15687235e-08,   1.15554278e-08]))
[ 129.        ,  -1.1705945 ,   1.        ,  -0.62827393,  -0.90087425]
(array([  1.65585426e-08,   1.65694052e-08,   1.65565207e-08]), array([  1.15975993e-08,   1.15843678e-08,   1.16017669e-08]))
[ 130.        ,  -1.42156903,   1.        ,  -0.8887984 ,  -1.1578308 ]
(array([  1.64570133e-08,   1.64443628e-08,   1.64315772e-08]), array([  1.16118781e-08,   1.16288087e-08,   1.16458712e-08]))
[ 131.        ,  -1.40544181,   1.        ,  -1.14540867,  -

KeyboardInterrupt: 